<div class='status' style='background-color: #183a4b; color: white; padding-top: 4px; padding-bottom: 4px; padding-left: 20px; padding-right: 20px; border-radius: 10px; font-family: Arial, sans-serif; font-size: 26px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.5);'><b>AI MedReview</B> - Fake GP Surgery Reviews | Processing Pipeline</div>

## Loading Libraries

In [1]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import datetime
import os
from tqdm import tqdm

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
warnings.filterwarnings("ignore")

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

**Loading Libraries** from AI MedReview Pipeline

In [2]:
from ai_medreview.fake_data_pipeline import *

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the ch

<div class='status' style='background-color: #f5c244; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Loading</b> dataset</div>

Loading 1/3 datasets chunks for processing

In [12]:
fake_data = pd.read_csv("../ai_medreview/data/fake_gp_reviews_llm_02.csv")
fake_data.columns = ["free_text", "generation_category"]
fake_data.head(2)

,free_text,generation_category
0,The GP practice is run by a friendly woman who...,Patient Respect
1,Staff were very friendly but slow. \n\nThe GP ...,Patient Respect


In [13]:
fake_data = word_count(fake_data)

2025-01-30 09:31:37.321 | INFO     | ai_medreview.utils:wrapper:15 - 🖥️    Started: 'word_count'
2025-01-30 09:31:37.348 | INFO     | ai_medreview.utils:wrapper:21 - ✅ Completed: 'word_count' ⚡️0.025217 sec


In [14]:
fake_data = sentiment_analysis(fake_data, "free_text")

2025-01-30 09:31:39.668 | INFO     | ai_medreview.fake_data_pipeline:sentiment_analysis:77 - 💛 Sentiment Analysis - Functions started.
█████████████████████████████████████| 1935/1935 [02:14<00:00, 14.44it/s]


In [15]:
fake_data = cleanup_neutral_sentiment(fake_data, "free_text")

2025-01-30 09:33:53.831 | INFO     | ai_medreview.fake_data_pipeline:cleanup_neutral_sentiment:162 - 🧻 Cleanup_neutral_sentiment - if free_text and do_better isna()


In [16]:
fake_data = feedback_classification(fake_data, batch_size=8)

2025-01-30 09:33:53.888 | INFO     | ai_medreview.utils:wrapper:15 - 🖥️    Started: 'feedback_classification'
Processing batches: 100%|██████████████████████████████████████| 242/242 [5:39:44<00:00, 84.23s/it]
2025-01-30 15:13:42.319 | INFO     | ai_medreview.utils:wrapper:21 - ✅ Completed: 'feedback_classification' ⚡️20388.427670 sec


In [17]:
fake_data.head(2)

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels
0,The GP practice is run by a friendly woman who...,Patient Respect,878,positive,0.873504,Reception Staff Interaction
1,Staff were very friendly but slow. \n\nThe GP ...,Patient Respect,88,positive,0.441180,Reception Staff Interaction
2,The GP surgery staff were kind and attentive d...,Patient Respect,385,positive,0.590509,Reception Staff Interaction
3,The staff were friendly and efficient making m...,Patient Respect,266,positive,0.922695,Reception Staff Interaction
4,The staff were friendly and accommodating.\nTh...,Patient Respect,116,positive,0.920581,Test Results
...,...,...,...,...,...,...
1930,The GP surgery staff were unfriendly and disor...,Administrative Efficiency,68,negative,0.897895,Reception Staff Interaction
1931,"The staff were very helpful and efficient, mak...",Administrative Efficiency,67,positive,0.878784,Administrative Efficiency
1932,Telehealth services were great but took hours ...,Telehealth,52,positive,0.611423,Telehealth
1933,The telehealth service was a lifesaver during ...,Telehealth,454,positive,0.909477,Telehealth


In [19]:
fake_data = emotion_classification(fake_data, "free_text", classifier=classifier)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the ch

NameError: name 'data' is not defined

In [20]:
fake_data.head(2)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the ch

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels,emotion_free_text
0,The GP practice is run by a friendly woman who...,Patient Respect,878,positive,0.873504,Reception Staff Interaction,disappointment
1,Staff were very friendly but slow. \n\nThe GP ...,Patient Respect,88,positive,0.441180,Reception Staff Interaction,disappointment


In [21]:
fake_data.to_csv('../ai_medreview/data/fake_gp_reviews_02_processed.csv', index=False)